In [2]:
from __future__ import print_function
import os
import numpy as np
from numpy import newaxis
from sklearn import preprocessing
np.random.seed(1337)

import keras
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Dropout
from keras.models import Model,Sequential
import sys

import pandas as pd
import random

Using TensorFlow backend.


# Reading in data

In [3]:
print("Data read started...")
data = pd.read_csv("result_nback_tetris.csv")
data = data.as_matrix()
print ("Data read finished.")

print(data.shape)

Data read started...
Data read finished.
(1125, 11)


# Eliminate EEG data

In [4]:
for i in range (1,5):
    data = np.delete(data, i, 1) 

data.shape

(1125, 7)

In [5]:
data[1]

array([3, 126910.0, 150630.0, 1839.9, 4.746279069767441, 371.67, 522.54], dtype=object)

# Z normalize

In [8]:
for i in range (1,7):
    data[:, i] = preprocessing.scale(data[:, i])

/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConve

In [9]:
x_data = data[:, 1:]
y_data = data[:, 0]

# One-hot encoding

In [10]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
y_one_hot = ohe.fit_transform(y_data.reshape(-1,1)).toarray()
print(y_one_hot)

[[ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 ..., 
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]]


# Shuffle data

In [11]:
indices = np.arange(x_data.shape[0])
np.random.shuffle(indices)

x_data = x_data[indices]
y_one_hot = y_one_hot[indices]

# Divide into train, validation and test data

In [12]:
len_data = len(x_data)

nb_test = int(len_data*0.15)
nb_validation = int(len_data*0.15)
nb_train = int(len_data*0.7)

end_valid = nb_train+nb_validation

x_train = x_data[0:nb_train]
y_train = y_one_hot[0:nb_train]

x_valid = x_data[nb_train:end_valid]
y_valid = y_one_hot[nb_train:end_valid]

x_test = x_data[end_valid:]
y_test = y_one_hot[end_valid:]

In [13]:
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape, x_test.shape, y_test.shape)

(787, 6) (787, 3) (168, 6) (168, 3) (170, 6) (170, 3)


# Build the net

In [14]:
model = Sequential()

model.add(Dense(200, input_shape=(6,)))
model.add(Dropout(0.25))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(3, activation='softmax'))

In [15]:
batch_size = 50
epochs = 150
earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, verbose=0, mode='auto')

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(x_train, y_train, nb_epoch=epochs,batch_size=batch_size, 
          callbacks=[earlyStopping], shuffle=True, validation_data = (x_valid, y_valid))

Train on 787 samples, validate on 168 samples
Epoch 1/150
787/787 [==============================] - 0s - loss: 1.0511 - acc: 0.4447 - val_loss: 1.0121 - val_acc: 0.5060
Epoch 2/150
787/787 [==============================] - 0s - loss: 0.9908 - acc: 0.5210 - val_loss: 0.9514 - val_acc: 0.5476
Epoch 3/150
787/787 [==============================] - 0s - loss: 0.9655 - acc: 0.5565 - val_loss: 0.9251 - val_acc: 0.6310
Epoch 4/150
787/787 [==============================] - 0s - loss: 0.9332 - acc: 0.5476 - val_loss: 0.8655 - val_acc: 0.5714
Epoch 5/150
787/787 [==============================] - 0s - loss: 0.8830 - acc: 0.5832 - val_loss: 0.8204 - val_acc: 0.6488
Epoch 6/150
787/787 [==============================] - 0s - loss: 0.8735 - acc: 0.5756 - val_loss: 0.7784 - val_acc: 0.6190
Epoch 7/150
787/787 [==============================] - 0s - loss: 0.8277 - acc: 0.6074 - val_loss: 0.7587 - val_acc: 0.6548
Epoch 8/150
787/787 [==============================] - 0s - loss: 0.8009 - acc: 0.6112

In [16]:
model.evaluate(x_test, y_test, batch_size=batch_size)

170/170 [==============================] - 0s     


[0.51653211257036991, 0.78235294187770166]

In [18]:
from sklearn.metrics import classification_report,confusion_matrix

y_pred = model.predict_classes(x_test)
print(y_pred)

p=model.predict_proba(x_test)

target_names = ['class 0(level1)', 'class 1(level2)', 'class 2(level3)']
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))
print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

160/170 [===========================>..] - ETA: 0s[2 0 2 2 0 2 0 0 2 1 0 1 1 0 2 2 0 2 0 2 0 0 2 0 0 2 2 0 2 0 1 0 2 0 1 2 0
 1 0 2 2 1 2 2 0 1 1 0 0 2 2 1 1 2 0 2 2 1 2 0 0 1 2 2 2 1 0 0 0 1 0 2 0 1
 2 2 2 2 1 2 2 0 2 1 1 0 0 2 2 2 1 1 2 0 1 2 0 0 0 2 0 0 1 1 2 0 2 0 0 0 2
 0 0 1 2 1 2 0 1 2 0 2 2 1 1 0 2 2 1 1 0 1 0 2 2 2 2 2 1 1 2 1 0 0 1 1 2 0
 2 0 2 0 0 2 1 2 1 1 0 2 2 1 2 0 2 0 0 0 1 0]
160/170 [===========================>..] - ETA: 0s                 precision    recall  f1-score   support

class 0(level1)       0.85      0.95      0.90        55
class 1(level2)       0.71      0.60      0.65        50
class 2(level3)       0.76      0.78      0.77        65

    avg / total       0.78      0.78      0.78       170

[[52  2  1]
 [ 5 30 15]
 [ 4 10 51]]
